## Final Project Submission

Please fill out:
* Student name: Amani Wanene
* Student pace:  full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


1. Import relevant resources

In [158]:
# Your code here - remember to use markdown cells for comments as well!
import pandas as pd
import os
import sqlite3
import matplotlib.pyplot as plt
import numpy as np


2. Import the datasets.

In [159]:
bom_df = pd.read_csv('zippedData/bom.movie_gross.csv.gz',encoding = 'latin1')
movie_info_df = pd.read_csv('zippedData/rt.movie_info.tsv.gz',delimiter = '\t')
reviews_df = pd.read_csv('zippedData/rt.reviews.tsv.gz',delimiter = '\t',encoding = 'latin1')
tmdb_df = pd.read_csv('zippedData/tmdb.movies.csv.gz')
movie_budgets_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')


In [160]:


import zipfile

# Extract IMDb SQL .db file
with zipfile.ZipFile('./zippedData/im.db.zip') as zipObj:
    # Extract all contents of .zip file into current directory
    zipObj.extractall(path='./zippedData/')
    
# Create connection to IMDb DB


In [161]:
imdb_conn = sqlite3.connect('zippedData/im.db')


In [162]:
imdb_df = pd.read_sql("""SELECT * FROM movie_basics
 JOIN movie_ratings
USING(movie_id)
""",imdb_conn)

3.View the datasets

In [163]:
print(bom_df.shape)
bom_df.isna().sum()




(3387, 5)


title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

In [164]:
print(movie_info_df.shape)
movie_info_df.isna().sum()

(1560, 12)


id                 0
synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64

In [165]:
print(reviews_df.shape)
reviews_df.isna().sum()

(54432, 8)


id                0
review         5563
rating        13517
fresh             0
critic         2722
top_critic        0
publisher       309
date              0
dtype: int64

In [166]:
tmdb_df.isna().sum()

Unnamed: 0           0
genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

In [167]:
movie_budgets_df.isna().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [168]:
print(imdb_df.shape)
imdb_df.isna().sum()

(73856, 8)


movie_id              0
primary_title         0
original_title        0
start_year            0
runtime_minutes    7620
genres              804
averagerating         0
numvotes              0
dtype: int64

4. Clean Data

A) bom_df

In [169]:
#drop rows with missing domestic_gross
bom_df = bom_df.dropna(subset=['domestic_gross'])

print(bom_df.loc[bom_df['domestic_gross'].isna()])

Empty DataFrame
Columns: [title, studio, domestic_gross, foreign_gross, year]
Index: []


In [170]:
#check percentage of foreign_gross missing values
bom_df['foreign_gross'].isna().sum()
missing_values_count = len(bom_df[bom_df['foreign_gross'].isna()])
total_rows = len(bom_df)

proportion_missing_values = missing_values_count / total_rows
print(proportion_missing_values)


0.40190532896695447


In [171]:
#Change null values to numeric in order to perform mathematical operations on the column.
bom_df['foreign_gross'] = pd.to_numeric(bom_df['foreign_gross'], errors='coerce')
bom_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3359 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3359 non-null   object 
 1   studio          3356 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2004 non-null   float64
 4   year            3359 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 157.5+ KB


In [172]:
#check the top 10 movies with missing foreign_domestic values
missing_foreign_gross_movies = bom_df[bom_df['foreign_gross'].isna()]
top_10_movies = missing_foreign_gross_movies.sort_values(by='domestic_gross', ascending=False).head(10)
print(top_10_movies)

                             title studio  domestic_gross  foreign_gross  year
1872  Star Wars: The Force Awakens     BV     936700000.0            NaN  2015
3079        Avengers: Infinity War     BV     678800000.0            NaN  2018
1873                Jurassic World   Uni.     652300000.0            NaN  2015
1874                     Furious 7   Uni.     353000000.0            NaN  2015
2760       The Fate of the Furious   Uni.     226000000.0            NaN  2017
3160                     Book Club   Par.      68600000.0            NaN  2018
1955                      War Room   TriS      67800000.0            NaN  2015
2864                All Eyez on Me   LG/S      44900000.0            NaN  2017
833                Red Dawn (2012)     FD      44800000.0            NaN  2012
2866                47 Meters Down  ENTMP      44300000.0            NaN  2017


In [173]:
#filter out movies with domestic gross < $10,000,000
filtered_movies = bom_df[bom_df['domestic_gross'] >= 10000000]
filtered_movies.isna().sum()

title              0
studio             0
domestic_gross     0
foreign_gross     58
year               0
dtype: int64

In [174]:
bom_df = filtered_movies

In [175]:
bom_df['gross_difference'] = bom_df['foreign_gross'] - bom_df['domestic_gross']
bom_df.head()

,title,studio,domestic_gross,foreign_gross,year,gross_difference
0,Toy Story 3,BV,415000000.0,652000000.0,2010,237000000.0
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010,357100000.0
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010,368300000.0
3,Inception,WB,292600000.0,535700000.0,2010,243100000.0
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010,275200000.0


In [179]:
average_difference = bom_df['gross_difference'].mean()
bom_df['foreign_gross'] = bom_df['foreign_gross'].fillna(bom_df['domestic_gross'] + average_difference)
bom_df = bom_df.drop(columns=['gross_difference'])



In [180]:
bom_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1170 entries, 0 to 3226
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           1170 non-null   object 
 1   studio          1170 non-null   object 
 2   domestic_gross  1170 non-null   float64
 3   foreign_gross   1170 non-null   float64
 4   year            1170 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 54.8+ KB


B) movie_info_df